## Introduction 

### REcommending the framework


In [13]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import time
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [14]:
n_months = 18
past_date = datetime.today().replace(day = 1) + relativedelta(months = -n_months)
past_financial_date = (past_date + relativedelta(months = -3)).strftime("%Y%m")
print(past_financial_date)

202301


In [15]:
# Connectin to the database

password = pd.read_csv(r'C:\Users\Naresh.Sampara\PycharmProjects\P9_FRAMEWORD_recommender\Password.csv')

DB_username = password.loc[password['Type']=='Database', 'UserName'].values[0]
DB_password = password.loc[password['Type']=='Database', 'password'].values[0]

# step 1 databse connection 
# Input details for the SQL database
DB_TYPE = "mssql"
DB_USER = DB_username
DB_PWD = DB_password
DB_SERVER = "azp-ukw-sql01.database.windows.net"
DB_PORT = "1433"
DB_NAME = "PBI"
DB_DRIVER = "SQL Server"
# Connect to the database
conn_string = '{}://{}:{}@{}:{}/{}?driver={}'.format(DB_TYPE, DB_USER, DB_PWD, DB_SERVER, DB_PORT, DB_NAME, DB_DRIVER)
from sqlalchemy import create_engine
engine = create_engine(conn_string)
# Open a connection
conn = engine.connect()

In [16]:
# Read recent spend data
sql = """SELECT DISTINCT "FYMonthKey", "CustomerURN", "FrameworkLotsID" FROM SpendAggregated WHERE "EvidencedSpend" > 0 AND "FYMonthKey" >= """ + past_financial_date
df_retrieval = pd.read_sql(sql, conn)
df_retrieval = df_retrieval.dropna()
df_retrieval['user'] = df_retrieval.pop('CustomerURN').astype(int).astype(str)
df_retrieval['item'] = df_retrieval.pop('FrameworkLotsID').astype(int).astype(str)
# Convert to calendar date
df_retrieval['date'] = pd.to_datetime(df_retrieval.pop('FYMonthKey'), format = "%Y%m") + pd.DateOffset(months = 3)

Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\Naresh.Sampara\PycharmProjects\P9_FRAMEWORD_recommender\.venv\lib\site-packages\sqlalchemy\pool\base.py", line 986, in _finalize_fairy
    fairy._reset(
  File "c:\Users\Naresh.Sampara\PycharmProjects\P9_FRAMEWORD_recommender\.venv\lib\site-packages\sqlalchemy\pool\base.py", line 1432, in _reset
    pool._dialect.do_rollback(self)
  File "c:\Users\Naresh.Sampara\PycharmProjects\P9_FRAMEWORD_recommender\.venv\lib\site-packages\sqlalchemy\dialects\mssql\base.py", line 3121, in do_rollback
    super().do_rollback(dbapi_connection)
  File "c:\Users\Naresh.Sampara\PycharmProjects\P9_FRAMEWORD_recommender\.venv\lib\site-packages\sqlalchemy\engine\default.py", line 699, in do_rollback
    dbapi_connection.rollback()
pyodbc.Error: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionWrite (send()). (10054) (SQLEndTran); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network 

In [17]:
print(df_retrieval.head())

       user   item       date
0  10588259  20598 2024-03-01
1  10027970  11319 2023-04-01
2  10015424  23161 2023-05-01
3  10038842   2274 2023-10-01
4  10598421   3709 2024-06-01


In [19]:
# Read all user spend data to get view history
sql = """SELECT DISTINCT "CustomerURN", "FrameworkLotsID" FROM SpendAggregated"""
user_seen = pd.read_sql(sql, conn)
user_seen = user_seen.dropna()
user_seen['user'] = user_seen.pop('CustomerURN').astype(int).astype(str)
user_seen['item'] = user_seen.pop('FrameworkLotsID').astype(int).astype(str)
user_seen = user_seen.groupby('user')['item'].apply(list)
user_seen = user_seen.apply(lambda x: list(set(x)))
user_seen = user_seen.to_dict()


{'0': ['11721', '910', '1339', '26419', '22998', '11662', '9414', '16027', '340', '16591', '26420', '19902', '11323', '14596', '16281', '21616', '19903', '19152', '2125', '18998', '14670', '19151', '20812', '15946', '1687', '20007', '22717', '18571', '14598', '11660', '1516', '1688', '20813', '1505', '2376', '2273', '2378', '11661', '5899', '23161'], '1': ['2481', '1505'], '10000006': ['16996', '17053', '405', '11319', '404', '2271', '340', '2237', '20378', '877', '11320', '16281', '2123', '17370', '10140', '17942', '20486', '878', '27660', '1985', '26530', '2176', '11651', '2177', '11322', '27657', '2101', '10330', '21992', '20598', '798', '27675'], '10000007': ['16591', '2486', '588'], '10000009': ['19903'], '10000010': ['272'], '10000014': ['2283', '2230'], '10000015': ['18285', '26419', '2481', '16996', '405', '11319', '404', '340', '16591', '2237', '26975', '751', '21938', '17852', '877', '11320', '401', '2123', '25962', '17370', '2163', '20486', '18284', '2274', '24293', '18888',

In [20]:
# Read in customers
sql = """SELECT DISTINCT "CustomerKey", "CustomerName", "MarketSector", "MarketSubSector" FROM sf.Attributes_sf_vw_Customers WHERE "Status" = 'Active' AND "MarketSector" IS NOT NULL AND "MarketSubSector" IS NOT NULL  AND "RecordExpiredDate" IS NULL"""
user_data = pd.read_sql(sql, conn)
user_data['user'] = user_data.pop('CustomerKey').astype(int).astype(str)
user_data = user_data.dropna()
user_data = user_data.drop_duplicates(subset = 'user')
# Remove where "Closed" appears in the customer name
user_data = user_data.loc[~user_data['CustomerName'].str.lower().str.contains('(closed)', regex = False)]
print(user_data.head())

                                    CustomerName MarketSector  \
0                    North East Scotland College    Education   
1                                Aboyne Hospital       Health   
2  Banff and Buchan College of Further Education    Education   
3                               Jubilee Hospital       Health   
4                       Robert Gordon University    Education   

                           MarketSubSector      user  
0  College of Further and Higher Education  10000006  
1    Community and Mental Health Providers  10000009  
2  College of Further and Higher Education  10000010  
3    Community and Mental Health Providers  10000014  
4                             Universities  10000015  


In [29]:
print('df retrieval')
print(df_retrieval.head(5))
print(df_retrieval.shape)
print('User data')
print(user_data.head(5))
print(user_data.shape)


df retrieval
       user   item       date
0  10588259  20598 2024-03-01
1  10027970  11319 2023-04-01
2  10015424  23161 2023-05-01
3  10038842   2274 2023-10-01
4  10598421   3709 2024-06-01
(648274, 3)
User data
                                    CustomerName MarketSector  \
0                    North East Scotland College    Education   
1                                Aboyne Hospital       Health   
2  Banff and Buchan College of Further Education    Education   
3                               Jubilee Hospital       Health   
4                       Robert Gordon University    Education   

                           MarketSubSector      user  
0  College of Further and Higher Education  10000006  
1    Community and Mental Health Providers  10000009  
2  College of Further and Higher Education  10000010  
3    Community and Mental Health Providers  10000014  
4                             Universities  10000015  
(52357, 4)
